In [1]:
import pandas as pd
import numpy as np
import selection
import weights
import btag
import root_pandas
import importlib

Welcome to JupyROOT 6.20/06


/eos/home-l/llayer/cmsopen/columnar/btag.py:13: RuntimeWarning: invalid value encountered in true_divide
  eff = np.divide(num[0], denom[0])


In [2]:
data = ['Run2011A_MultiJet', 'Run2011B_MultiJet']
mc = ['T_TuneZ2_s', 'WJetsToLNu', 'DYJetsToLL', 'T_TuneZ2_tW', 'T_TuneZ2_t-channel',
       'Tbar_TuneZ2_s', 'Tbar_TuneZ2_tW', 'Tbar_TuneZ2_t-channel', 'TTJets']

In [ ]:
import ROOT
f = ROOT.TFile()

In [21]:
from coffea.lookup_tools import extractor

In [23]:
ext = extractor()
ext.add_weight_sets([
    "* * data/Legacy11_V1_DATA_UncertaintySources_AK5PFchs.junc.txt",
    "* * jer_sf.root",
    "* * JESUncMC.root"
])
ext.finalize()

evaluator = ext.make_evaluator()

In [156]:
ext = extractor()
ext.add_weight_sets([
    "jet40_ * rootFilesTurnOn/TriggerEffHisto_data_match40_JETLEG.root",
    "jet45_ * rootFilesTurnOn/TriggerEffHisto_data_match45_JETLEG.root",
    "tau40_ * rootFilesTurnOn/TriggerEffHisto_match40_newTauID.root",
    "tau45_ * rootFilesTurnOn/TriggerEffHisto_match45_newTauID.root"
])
ext.finalize()

evaluator = ext.make_evaluator()

In [157]:
evaluator.keys()

dict_keys(['jet40_jet1_ref', 'jet40_jet1_ref_error', 'jet40_jet2_ref', 'jet40_jet2_ref_error', 'jet40_jet3_ref', 'jet40_jet3_ref_error', 'jet40_jet4_ref', 'jet40_jet4_ref_error', 'jet40_jet1_trig', 'jet40_jet1_trig_error', 'jet40_jet2_trig', 'jet40_jet2_trig_error', 'jet40_jet3_trig', 'jet40_jet3_trig_error', 'jet40_jet4_trig', 'jet40_jet4_trig_error', 'jet40_jet1_eff', 'jet40_jet1_eff_error', 'jet40_jet2_eff', 'jet40_jet2_eff_error', 'jet40_jet3_eff', 'jet40_jet3_eff_error', 'jet40_jet4_eff', 'jet40_jet4_eff_error', 'jet40_reference_tau', 'jet40_reference_tau_error', 'jet40_triggered_tau', 'jet40_triggered_tau_error', 'jet40_eff_tau', 'jet40_eff_tau_error', 'jet40_reference_tau_eta', 'jet40_reference_tau_eta_error', 'jet40_triggered_tau_eta', 'jet40_triggered_tau_eta_error', 'jet40_eff_tau_eta', 'jet40_eff_tau_eta_error', 'jet40_HLTtau_pt', 'jet40_HLTtau_pt_error', 'jet40_HLTtau_eta', 'jet40_HLTtau_eta_error', 'jet40_HLTtau_pt_diff', 'jet40_HLTtau_pt_diff_error', 'jet40_HLTtau_eta_dif

In [165]:
def eval_weight(obj, key):
    
    weight = evaluator[key](obj)
    mask = weight == 0.
    return np.where(mask, 1., weight)

In [172]:
def calc_weight(jet, tau, prefix):
    
    jet0 = eval_weight(jet.pt[:,0], "jet" + prefix + "_jet4_eff" )
    jet1 = eval_weight(jet.pt[:,1], "jet" + prefix + "_jet4_eff" )
    jet2 = eval_weight(jet.pt[:,2], "jet" + prefix + "_jet4_eff" )
    tau0 = eval_weight(tau.pt[:,0], "tau" + prefix + "_eff_tau" )
    
    jet0_err = eval_weight(jet.pt[:,0], "jet" + prefix + "_jet4_eff_error" )
    jet1_err = eval_weight(jet.pt[:,1], "jet" + prefix + "_jet4_eff_error" )
    jet2_err = eval_weight(jet.pt[:,2], "jet" + prefix + "_jet4_eff_error" )
    tau0_err = eval_weight(tau.pt[:,0], "tau" + prefix + "_eff_tau_error" )
    
    weight = 1. * jet0 * jet1 * jet2 * tau0
    weight_up = 1. * (jet0 + jet0_err) * (jet1 + jet1_err) * (jet2 + jet2_err) * (tau0 + tau0_err)
    weight_down = 1. * (jet0 - jet0_err) * (jet1 - jet1_err) * (jet2 - jet2_err) * (tau0 - tau0_err)
    
    return weight, weight_up, weight_down

In [178]:
np.random.seed(0)

def trigger_weight(jet, tau, frac=0.218):
    
    # Randomly select 40/45 trigger according to lumi fraction
    rand = np.random.random((len(jet)))
    mask_is40 = rand < frac
    
    weight_40, weight_up_40, weight_down_40 = calc_weight(jet, tau, "40")
    weight_45, weight_up_45, weight_down_45 = calc_weight(jet, tau, "45")
    
    weight = np.where(mask_is40, weight_40, weight_45)
    weight_up = np.where(mask_is40, weight_up_40, weight_up_45)
    weight_down = np.where(mask_is40, weight_down_45, weight_down_45)
    
    return weight, weight_up, weight_down
    

In [179]:
weight = trigger_weight(jet, tau)

In [180]:
f = ROOT.TFile("data/beff.root")
h2_BTaggingEff_Denom_b = f.Get("h2_BTaggingEff_Denom_b")
h2_BTaggingEff_Num_b = f.Get("h2_BTaggingEff_Num_b")
h2_BTaggingEff_Denom_c = f.Get("h2_BTaggingEff_Denom_c")
h2_BTaggingEff_Num_c = f.Get("h2_BTaggingEff_Num_c")
h2_BTaggingEff_Denom_usdg = f.Get("h2_BTaggingEff_Denom_udsg")
h2_BTaggingEff_Num_usdg = f.Get("h2_BTaggingEff_Num_udsg")

In [183]:
def to_eff(h1, h2):
    
    h3 = h1.Clone("h3")
    h3.Sumw2()
    h3.Divide(h2)
    return h3

In [209]:
outfile = ROOT.TFile("data/beff_precalc.root", "RECREATE")
b = to_eff(h2_BTaggingEff_Num_b, h2_BTaggingEff_Denom_b)
c = to_eff(h2_BTaggingEff_Num_c, h2_BTaggingEff_Denom_c)
usdg = to_eff(h2_BTaggingEff_Num_usdg, h2_BTaggingEff_Denom_usdg)
b.Write("b")
c.Write("c")
usdg.Write("usdg")
outfile.Close()

In [210]:
ff = ROOT.TFile("data/beff_precalc.root")

In [191]:
ext = extractor()
ext.add_weight_sets([
    "* * data/beff_precalc.root"
])
ext.finalize()

evaluator = ext.make_evaluator()

In [195]:
evaluator.keys()

dict_keys(['b', 'b_error', 'c', 'c_error', 'usdg', 'usdg_error'])

In [207]:
import root_numpy

In [217]:
h1 = ff.Get("b")
h2 = ff.Get("c")
h3 = ff.Get("usdg")

In [242]:
edges

[array([   0.,   10.,   20.,   30.,   40.,   50.,   60.,   70.,   80.,
          90.,  100.,  110.,  120.,  130.,  140.,  150.,  160.,  170.,
         180.,  190.,  200.,  210.,  220.,  230.,  240.,  250.,  260.,
         270.,  280.,  290.,  300.,  310.,  320.,  330.,  340.,  350.,
         360.,  370.,  380.,  390.,  400.,  410.,  420.,  430.,  440.,
         450.,  460.,  470.,  480.,  490.,  500.,  510.,  520.,  530.,
         540.,  550.,  560.,  570.,  580.,  590.,  600.,  610.,  620.,
         630.,  640.,  650.,  660.,  670.,  680.,  690.,  700.,  710.,
         720.,  730.,  740.,  750.,  760.,  770.,  780.,  790.,  800.,
         810.,  820.,  830.,  840.,  850.,  860.,  870.,  880.,  890.,
         900.,  910.,  920.,  930.,  940.,  950.,  960.,  970.,  980.,
         990., 1000.]),
 array([-3. , -2.9, -2.8, -2.7, -2.6, -2.5, -2.4, -2.3, -2.2, -2.1, -2. ,
        -1.9, -1.8, -1.7, -1.6, -1.5, -1.4, -1.3, -1.2, -1.1, -1. , -0.9,
        -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.

In [218]:
b_num, edges = root_numpy.hist2array(h1, return_edges=True)
c_num, _ = root_numpy.hist2array(h2, return_edges=True)
usdg_num, _ = root_numpy.hist2array(h3, return_edges=True)

In [220]:
c_num.shape

(100, 60)

In [225]:
sf = np.dstack((b_num, c_num, usdg_num))

In [226]:
sf.shape

(100, 60, 3)

In [227]:
sf[jet.pt, jet.eta, 0]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [254]:
hist = ROOT.TH3F("hist", "hist", 100, 0., 1000., 60, -3., 3., 3, 0, 3)

Warning in <TFile::Append>: Replacing existing TH1: hist (Potential memory leak).


In [255]:
from root_numpy import array2hist, hist2array

In [256]:
_ = array2hist(sf, hist)

In [257]:
outtest = ROOT.TFile("data/beff_test.root", "RECREATE")
hist.Write("eff")
outtest.Close()

In [258]:
fff = ROOT.TFile("data/beff_test.root")

In [259]:
fff.ls()

TFile**		data/beff_test.root	
 TFile*		data/beff_test.root	
  KEY: TH3F	eff;1	hist


In [260]:
ext = extractor()
ext.add_weight_sets([
    "* * data/beff_test.root"
])
ext.finalize()

evaluator = ext.make_evaluator()

In [261]:
evaluator.keys()

dict_keys(['eff', 'eff_error'])

In [262]:
evaluator["eff"]

3 dimensional histogram with axes:
	1: [   0.   10.   20.   30.   40.   50.   60.   70.   80.   90.  100.  110.
  120.  130.  140.  150.  160.  170.  180.  190.  200.  210.  220.  230.
  240.  250.  260.  270.  280.  290.  300.  310.  320.  330.  340.  350.
  360.  370.  380.  390.  400.  410.  420.  430.  440.  450.  460.  470.
  480.  490.  500.  510.  520.  530.  540.  550.  560.  570.  580.  590.
  600.  610.  620.  630.  640.  650.  660.  670.  680.  690.  700.  710.
  720.  730.  740.  750.  760.  770.  780.  790.  800.  810.  820.  830.
  840.  850.  860.  870.  880.  890.  900.  910.  920.  930.  940.  950.
  960.  970.  980.  990. 1000.]
	2: [-3.  -2.9 -2.8 -2.7 -2.6 -2.5 -2.4 -2.3 -2.2 -2.1 -2.  -1.9 -1.8 -1.7
 -1.6 -1.5 -1.4 -1.3 -1.2 -1.1 -1.  -0.9 -0.8 -0.7 -0.6 -0.5 -0.4 -0.3
 -0.2 -0.1  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.   1.1
  1.2  1.3  1.4  1.5  1.6  1.7  1.8  1.9  2.   2.1  2.2  2.3  2.4  2.5
  2.6  2.7  2.8  2.9  3. ]
	3: [0. 1. 2. 3.]

In [265]:
evaluator["eff"](100., 0.15, 2.)

0.012711762

In [270]:
import copy

In [275]:

for j, t in zip(jet, tau):
    
    p4 = copy.deepcopy(t["p4"][0])
    print("Tau p4", p4)
    HT, H = 0., 0.
    for vec in j:
        p4 += vec["p4"]
    print("Added", p4)
    #print( j["p4"].pt, t["p4"].pt )

Tau p4 TLorentzVector(51.164, -0.59398, 1.7806, 0.87758)
Added TLorentzVector(-10.796, -59.256, -115.75, 571.68)
Tau p4 TLorentzVector(45.569, 2.2877, -2.4951, 0.81105)
Added TLorentzVector(9.0903, 46.581, 845.48, 1143.7)
Tau p4 TLorentzVector(67.135, 0.97985, 0.10675, 1.2852)
Added TLorentzVector(-67.808, 78.609, 294.41, 1345.4)
Tau p4 TLorentzVector(48.314, 0.89961, 1.1036, 0.13957)
Added TLorentzVector(-0.61799, 14.496, 82.415, 435.71)
Tau p4 TLorentzVector(76.19, -0.64243, -2.304, 1.082)
Added TLorentzVector(-8.1471, 4.62, -1231.4, 1386.4)
Tau p4 TLorentzVector(56.506, 0.52762, -2.8298, 0.53326)
Added TLorentzVector(3.5307, 9.2098, 145.16, 389.05)
Tau p4 TLorentzVector(74.513, 0.22192, -2.2742, 0.62249)
Added TLorentzVector(36.168, -6.963, 115.37, 456.52)
Tau p4 TLorentzVector(131.81, -0.34514, 0.21226, 0.13957)
Added TLorentzVector(-108.85, -9.6578, -70.147, 797.66)
Tau p4 TLorentzVector(76.648, 0.15379, 0.17856, 0.99302)
Added TLorentzVector(-306.62, -87.184, -582.58, 1310.6)
Tau

In [205]:
jet["p4"][0]

<PtEtaPhiMassLorentzVectorArray [TLorentzVector(163.2, -0.019414, 0.40543, 16.03) TLorentzVector(147.07, -0.5662, -2.1255, 17.739) TLorentzVector(87.174, 0.60444, -3.1305, 15.547) TLorentzVector(49.84, -0.86283, -1.2753, 9.7447)] at 0x7fc9d4bb41d0>

In [206]:
sum(jet["p4"][0])

TypeError: cannot add a TLorentzVector with a int

In [203]:
evaluator["usdg"](jet.pt, jet.eta)

ValueError: starts and stops are not compatible with a single offsets array

In [31]:
evaluator.keys()

dict_keys(['jet40', 'jet45', 'tau40', 'tau45'])

In [32]:
evaluator["jet40"](100.)

0.9672131

In [201]:
importlib.reload(selection)
path = "/eos/user/l/llayer/opendata_files/preselection_merged/" + "T_TuneZ2_s" + ".root"
df, jet, tau, cut_flow = selection.event_selection(path, isData = False, isTT = False, corrLevel = "centJER")

Processing: T_TuneZ2_s.root isData: False isTT: False corrLevel centJER
SF 1.0 1.0


In [59]:
jet.pt[:,0]

array([164.9741  , 227.10472 , 298.7569  ,  99.604546, 139.6582  ,
        79.38644 ,  89.59161 , 343.78494 , 447.89755 , 118.52118 ,
       119.038055, 169.07281 , 189.04832 ,  98.94595 , 109.097916,
       194.0468  , 172.73466 , 150.04248 , 143.88855 , 172.40742 ,
       100.71692 , 117.78187 ,  97.741745, 178.94006 ,  63.02036 ,
       110.818504, 212.15852 , 242.76344 , 132.96727 , 106.133865],
      dtype=float32)

In [80]:
jet40 = evaluator["jet40"](jet.pt[:,0])

In [81]:
jet40

array([1.        , 0.        , 0.        , 0.94067794, 0.9166667 ,
       0.90782124, 0.93877554, 0.        , 0.        , 0.93333334,
       0.93333334, 1.        , 1.        , 0.94067794, 0.9672131 ,
       0.        , 1.        , 1.        , 1.        , 1.        ,
       0.9672131 , 0.93333334, 0.94067794, 1.        , 0.89365673,
       0.93333334, 0.        , 0.        , 0.9166667 , 0.9672131 ],
      dtype=float32)

In [82]:
mask = jet40 == 0.

In [83]:
mask

array([False,  True,  True, False, False, False, False,  True,  True,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False,  True,
        True, False, False])

In [87]:
jet40 = np.where(mask, 1., jet40)

In [88]:
jet40

array([1.        , 1.        , 1.        , 0.94067794, 0.9166667 ,
       0.90782124, 0.93877554, 1.        , 1.        , 0.93333334,
       0.93333334, 1.        , 1.        , 0.94067794, 0.9672131 ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.9672131 , 0.93333334, 0.94067794, 1.        , 0.89365673,
       0.93333334, 1.        , 1.        , 0.9166667 , 0.9672131 ],
      dtype=float32)

In [ ]:
def trigger_weight(jet, tau):
    
    

In [42]:
t = np.random.random((len(jet)))

In [45]:
t < 0.218

array([ True, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False,  True, False,
        True, False, False])

In [49]:
tt = np.where(t < 0.218, 0.7, 0.3)

In [50]:
tt

array([0.7, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.7, 0.3, 0.3, 0.3, 0.3,
       0.3, 0.3, 0.3, 0.3, 0.3, 0.7, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.7,
       0.3, 0.7, 0.3, 0.3])

In [ ]:
tt = np.where(t < 0.218, 0.7, 0.3)

In [11]:
importlib.reload(selection)
def event_selection():
    
    samples = {}
    for sample in mc + data:
        
        #!!!!!!! Careful with JER application before Tau?!
        
        if "TTJets" in sample: isTT = True
        else: isTT = False
            
        if "Run2011" in sample: isData = True
        else: isData = False
        
        path = "/eos/user/l/llayer/opendata_files/preselection_merged/" + sample + ".root"
        
        df, cut_flow = selection.event_selection(path, isData = isData, isTT = isTT)
        
        print( cut_flow )
        """
        df.to_hdf(outpath + sample + ".h5", "central", mode='w')
                
        if isData == False:
            
            for c in corrections:
                df, cut_flow = selection.event_selection(path, isData = isData, isTT = isTT, corrLevel = c)
                print( cut_flow )
                df.to_hdf(outpath + sample + ".h5", c, mode='a')  
        """
        
        samples[sample] = df
        
    return samples

samples = event_selection()

Processing: T_TuneZ2_s.root isData: False isTT: False corrLevel cent
{'preselected': 1915, 'hlt': 273, 'lep_veto': 190, 'jet_requirement': 114, 'tau_requirement': 30}
Processing: WJetsToLNu.root isData: False isTT: False corrLevel cent
{'preselected': 44906, 'hlt': 4982, 'lep_veto': 2385, 'jet_requirement': 1083, 'tau_requirement': 231}
Processing: DYJetsToLL.root isData: False isTT: False corrLevel cent
{'preselected': 114807, 'hlt': 19961, 'lep_veto': 5461, 'jet_requirement': 2265, 'tau_requirement': 547}
Processing: T_TuneZ2_tW.root isData: False isTT: False corrLevel cent
{'preselected': 20110, 'hlt': 5529, 'lep_veto': 3046, 'jet_requirement': 1698, 'tau_requirement': 568}
Processing: T_TuneZ2_t-channel.root isData: False isTT: False corrLevel cent
{'preselected': 23527, 'hlt': 3396, 'lep_veto': 2404, 'jet_requirement': 1203, 'tau_requirement': 240}
Processing: Tbar_TuneZ2_s.root isData: False isTT: False corrLevel cent
{'preselected': 1117, 'hlt': 158, 'lep_veto': 115, 'jet_requir

In [12]:
importlib.reload(btag)
importlib.reload(weights)

def candidates(sample, df, invert_btag = False, njets=-1):
    
    if "Run2011" in sample: isData = True
    else: isData = False

    print( "Processing:", sample,"isData:", isData, "invert_btag:", invert_btag)

    df['nJets'] = df["Jet_pt"].str.len()
    
    # b-tagging
    df["Jet_nbtags"] = df["Jet_csvDisc"].apply( lambda x : btag.count_btags(x, njets=njets) )
    if invert_btag:
        df = btag.no_tag(df)
    else:
        df = btag.at_least_1tag(df)

    # MET cut
    df = selection.met_requirement(df)

    
    # HL features
    #df = pd.concat([df, df.apply(lambda ev : pd.Series(hl.hlFeatures(ev, njets=njets)), axis=1)], axis=1)
    

    # MC weights
    if not isData:

        hlt_40, hlt_45 = weights.lumi()
        total_lumi = hlt_40 + hlt_45
        trigger_frac = hlt_40 / float(hlt_45)
        df = pd.concat([df, df.apply(lambda ev: pd.Series(btag.eval_sf_eff(ev)), axis=1)], axis=1)
        df["Jet_btag_weight1"] = df.apply(lambda ev : btag.b_weight_method1(ev, njets=njets), axis=1)
        #df["Jet_btag_weight2"] = df.apply(lambda ev : btag.b_weight_method2(ev, njets=njets), axis=1)
        # trigger weights
        #df["trigger_weight"] = df.apply(lambda ev : weights.trigger_weight(ev, trigger_frac), axis=1)
        df = pd.concat([df, df.apply(lambda ev: pd.Series(weights.trigger_weight(ev, trigger_frac)), axis=1)], axis=1)
        # normalization
        counts_path = "/eos/user/l/llayer/opendata_files/preselection_merged/" + sample + "_counts.root"
        total_counts = root_pandas.read_root(counts_path)
        xsec = weights.get_xsec(sample)
        weights.norm(df, total_counts, xsec, lumi = total_lumi)

    # QCD
    if isData & invert_btag:
        
        # Assume light flavour
        def lf(nJets):
            return np.zeros((nJets))
        df["Jet_flavour"] = df["nJets"].apply(lf)
        df = pd.concat([df, df.apply(lambda ev: pd.Series(btag.eval_sf_eff(ev)), axis=1)], axis=1)
        df["btag_weight"] = df.apply(lambda ev : btag.b_weight_method2(ev, njets=njets), axis=1)
        
    return df

/eos/home-l/llayer/cmsopen/columnar/btag.py:13: RuntimeWarning: invalid value encountered in true_divide
  eff = np.divide(num[0], denom[0])


In [13]:
def rearrange_samples(samples):
    
    # Concat and split MC in signal and background
    new_samples = {}
    new_samples["TTJets_signal"], new_samples["TTJets_bkg"] = weights.classify_tt(samples["TTJets"])
    new_samples["WZJets"] = pd.concat([samples['WJetsToLNu'], samples['DYJetsToLL']], axis=0)
    new_samples["STJets"] = pd.concat([samples['T_TuneZ2_s'], samples['T_TuneZ2_tW'], samples['T_TuneZ2_t-channel'], 
                              samples['Tbar_TuneZ2_s'], samples['Tbar_TuneZ2_tW'], 
                              samples['Tbar_TuneZ2_t-channel']], axis=0)

    # Concat the data
    new_samples["Data"] = pd.concat([samples["Run2011A_MultiJet"], samples["Run2011B_MultiJet"]], axis=0)
    new_samples["QCD"] = pd.concat([samples["QCD_Run2011A_MultiJet"], samples["QCD_Run2011B_MultiJet"]], axis=0)
    
    return new_samples

In [14]:
def proc_candidates(samples, njets = -1):
    
    cand_samples = {}
    for sample in data:
        cand_samples[sample] = candidates(sample, samples[sample], invert_btag = False, njets=njets)
        cand_samples["QCD_" + sample] = candidates(sample, samples[sample], invert_btag = True, njets=njets)
    
    
    for sample in mc:
        cand_samples[sample] = candidates(sample, samples[sample], invert_btag = False, njets=njets)   

    new_samples = rearrange_samples(cand_samples)
    return new_samples

In [15]:
cands = proc_candidates(samples)

Processing: Run2011A_MultiJet isData: True invert_btag: False
Processing: Run2011A_MultiJet isData: True invert_btag: True
Processing: Run2011B_MultiJet isData: True invert_btag: False
Processing: Run2011B_MultiJet isData: True invert_btag: True
Processing: T_TuneZ2_s isData: False invert_btag: False
Processing: WJetsToLNu isData: False invert_btag: False
Processing: DYJetsToLL isData: False invert_btag: False
Processing: T_TuneZ2_tW isData: False invert_btag: False
Processing: T_TuneZ2_t-channel isData: False invert_btag: False
Processing: Tbar_TuneZ2_s isData: False invert_btag: False
Processing: Tbar_TuneZ2_tW isData: False invert_btag: False
Processing: Tbar_TuneZ2_t-channel isData: False invert_btag: False
Processing: TTJets isData: False invert_btag: False


In [16]:
len(cands["Data"])

3238

In [18]:
import ROOT
from root_numpy import fill_hist

def save_var(sample, name, var_name, bins = 20, xlow = 0., xup = 350):

    hist = ROOT.TH1D(name + "_" + var_name, name + "_" + var_name, bins, xlow, xup)
    hist.Sumw2()
    
    if name == "Data":
        pass
    elif name == "QCD":
        if var_name == "bdt":
            scale_qcd = 9.
            sample = sample[sample["train_flag"] == "test"]
        else:
            scale_qcd = 4.3
        sample['weight'] = sample['btag_weight'] * scale_qcd
    else:
        #samples[sample]['new_trigger_weight'] = new_samples[sample].apply(lambda ev : weights.trigger_weight(ev), axis=1)
        sample['weight'] = sample['norm'] * (1/1000) * sample['trigger_weight'] * sample['Jet_btag_weight1']
        #print(sample, sum(samples[sample]['weight']))
        #new_samples[sample]['btag_weight2']
    
    # Flatten if the column is a list
    if "Jet_" in var_name:
        series = sample[var_name].apply(pd.Series).stack().reset_index(drop=True)
        if name != "Data":
            sample['weight_stacked'] = sample.apply(lambda x : stack_weight(x["weight"] ,x["nJets"]), axis=1)
            weights = sample['weight_stacked'].apply(pd.Series).stack().reset_index(drop=True)
    else:
        series = sample[var_name]
        if name != "Data":
            weights = sample["weight"]
    if name == "Data":
        fill_hist(hist, series)
    else:
        fill_hist(hist, series, weights = weights)
    #hist.Write()
    return hist

def vars_to_histos(samples, variables):
    
    hists = {}
    for name, sample in samples.items():
        for var in variables:
            hists[name + "_" + var["var_name"]] = save_var(sample, name, var["var_name"], var["bins"], var["xlow"], var["xup"])

    return hists

In [19]:
variables = [
    {"var_name" : "MET_met", "bins" : 30, "xlow" : 0., "xup" : 400, "xtitle" : "MET [GeV]"}
]

hists = vars_to_histos(cands, variables)

In [20]:
f = ROOT.TFile("histos_test.root", "RECREATE")
for s, hist in hists.items():

    print( s, hist.Integral() )
    hist.Write()
    
f.Close()

TTJets_signal_MET_met 345.9627057153264
TTJets_bkg_MET_met 145.5093453003518
WZJets_MET_met 61.43496068440135
STJets_MET_met 30.534209478997937
Data_MET_met 3238.0
QCD_MET_met 2731.115160766783


In [76]:
def fit(histos, fit_var, corr = "central"):
        

    bkg = histos['TTJets_bkg' + "_" + fit_var ].Clone()
    bkg.Add(histos['WZJets' + "_" + fit_var ])
    bkg.Add(histos['STJets' + "_" + fit_var ])

    data = histos["Data" + "_" + fit_var ].Clone()
    data.Add(bkg, -1.)

    signal = histos["TTJets_signal" + "_" + fit_var ]

    qcd = histos["QCD" + "_" + fit_var ]

    x = ROOT.RooRealVar("x","x",0.,350.)
    rooSignal =  ROOT.RooDataHist("signal","signal",ROOT.RooArgList( x ), signal)
    rooBkg = ROOT.RooDataHist("bkg","bkg", ROOT.RooArgList( x ), qcd)
    signal_pdf = ROOT.RooHistPdf("signal","signal",ROOT.RooArgSet( x ), rooSignal)
    bkg_pdf = ROOT.RooHistPdf("bkg","bkg",ROOT.RooArgSet( x ), rooBkg)

    c0 = ROOT.RooRealVar("c0","c0",0.5,0.,1.)
    pdf = ROOT.RooAddPdf("pdf","pdf", signal_pdf,bkg_pdf, c0)

    dataFit = ROOT.RooDataHist("data","data",ROOT.RooArgList( x ), data);

    fitResult = pdf.fitTo(dataFit)
    #print( fitResult )

    sf_tt_sig = (c0.getVal() * data.Integral()) / signal.Integral() 
    sf_tt_sig_err = (c0.getError() * data.Integral()) / signal.Integral()
    
    sf_qcd = ((1-c0.getVal())*data.Integral()) / qcd.Integral()
    sf_qcd_err = (c0.getError() * data.Integral()) / qcd.Integral()
    
    
    print( "scale factor TTbar tau(h) QQ ", sf_tt_sig, "+-", sf_tt_sig_err )
    print( "scale factor MultiJet ", sf_qcd, "+-", sf_qcd_err )

    return sf_tt_sig, sf_qcd

In [77]:
fit(hists, "MET_met")

scale factor TTbar tau(h) QQ  0.9910458961661615 +- 0.09467095587401728
scale factor MultiJet  0.97667607748668 +- 0.011718521532530633


(0.9910458961661615, 0.97667607748668)

[#1] INFO:DataHandling -- RooDataHist::adjustBinning(signal): fit range of variable x expanded to nearest bin boundaries: [0,350] --> [0,360]
[#0] WARNING:InputArguments -- RooAbsPdf::fitTo(pdf) WARNING: a likelihood fit is requested of what appears to be weighted data.
       While the estimated values of the parameters will always be calculated taking the weights into account,
       there are multiple ways to estimate the errors of the parameters. You are advised to make an 
       explicit choice for the error calculation:
           - Either provide SumW2Error(true), to calculate a sum-of-weights-corrected HESSE error matrix
             (error will be proportional to the number of events in MC).
           - Or provide SumW2Error(false), to return errors from original HESSE error matrix
             (which will be proportional to the sum of the weights, i.e., a dataset with <sum of weights> events).
           - Or provide AsymptoticError(true), to use the asymptotically correct 